### CrewAI

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain_core.tools import tool
from duckduckgo_search import DDGS
from langchain.callbacks.tracers import ConsoleCallbackHandler

from crewai.tools import BaseTool
from langchain_experimental.utilities import PythonREPL


class CustomSearchTool(BaseTool):
    name: str = "Search"
    description: str = "Runs your query like a google search"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return DDGS().text(query, max_results=7)


search = CustomSearchTool()

repl = PythonREPL()


class CustomCodeExecutorTool(BaseTool):
    name: str = "CodeExecutor"
    description: str = "Runs your python code"

    def _run(self, code: str) -> str:
        "Executes your code"
        try:
            result = repl.run(code)
            print("RESULT OF CODE EXECUTION", result)
        except BaseException as e:
            return f"Failed to execute. Error: {repr(e)}"
        return result


code_executor = CustomCodeExecutorTool()

In [ ]:
from crewai import Agent, Task, Crew

searcher = Agent(
    llm=llm,
    role="Searcher",
    goal="Provide info gathered on the internet",
    backstory="",
    tools=[search],
    verbose=True,
)

engineer = Agent(
    llm=llm,
    role="Engineer",
    goal="Make calculations",
    backstory="",
    tools=[code_executor],
    verbose=True,
)

search_task = Task(
    description="Search the web for information",
    expected_output="Web search results",
    agent=searcher,
)
code_execution_task = Task(
    description="Run code", expected_output="Code execution output", agent=engineer
)

In [5]:
crew = Crew(
    agents=[searcher, engineer],
    tasks=[search_task, code_execution_task],
    verbose=True,
    planning=True,  # Enable planning feature
)

In [7]:
crew.kickoff(inputs={"request": "What's the current age of the pope, power 0.82?"})

 
[2024-12-03 13:43:46][INFO]: Planning the crew execution
# Agent: Searcher
## Task: Search the web for information1. **Initiate Task**  
   - The Searcher opens the designated platform for performing searches.

2. **Define the Search Query**  
   - The Searcher identifies the specific information required. For example, "latest AI technologies" or "research on climate change". 

3. **Utilize the Search Tool**  
   - The Searcher invokes the `CustomSearchTool` named 'Search'.  
   - Input the search query into the tool, following the format:  
     ```python  
     {  
         'query': {'description': "latest AI technologies", 'type': 'str'}  
     }  
     ```

4. **Execute the Web Search**  
   - After inputting the query, the Searcher runs the search.  
   - The tool will perform a web search similar to a Google search.

5. **Analyze Search Results**  
   - Once the search results are generated, the Searcher reviews the findings to extract pertinent information.  
   - Key details 

Python REPL can execute arbitrary code. Use with caution.


RESULT OF CODE EXECUTION NameError("name 'factorial' is not defined")


# Agent: Engineer
## Thought: To complete the task, I need to execute the provided Python code to calculate the factorial of a number. The code snippet provided is:
```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)
result = factorial(5)  # Expected output for 5!
```
I will use the CodeExecutor tool to run this code and obtain the result.
## Using tool: CodeExecutor
## Tool Input: 
"{\"code\": \"def factorial(n):\\n    if n == 0:\\n        return 1\\n    else:\\n        return n * factorial(n - 1)\\n\\nresult = factorial(5)\\nresult\"}"
## Tool Output: 
NameError("name 'factorial' is not defined")


# Agent: Engineer
## Thought: It seems there was an error in the code execution. The error message indicates that the function `factorial` is not recognized, which is unexpected given the code provided. I will attempt to execute the code again to ensure it runs corr

CrewOutput(raw='120', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Search the web for information1. **Initiate Task**  \n   - The Searcher opens the designated platform for performing searches.\n\n2. **Define the Search Query**  \n   - The Searcher identifies the specific information required. For example, "latest AI technologies" or "research on climate change". \n\n3. **Utilize the Search Tool**  \n   - The Searcher invokes the `CustomSearchTool` named \'Search\'.  \n   - Input the search query into the tool, following the format:  \n     ```python  \n     {  \n         \'query\': {\'description\': "latest AI technologies", \'type\': \'str\'}  \n     }  \n     ```\n\n4. **Execute the Web Search**  \n   - After inputting the query, the Searcher runs the search.  \n   - The tool will perform a web search similar to a Google search.\n\n5. **Analyze Search Results**  \n   - Once the search results are generated, the Searcher reviews the findings to extract pertine

Just tried running my crew without tasks, get:
`ValueError: Something went wrong. Kickoff should return only one task output.`


Maybe I should put tasks.

UPDATE: ok, after adding tasks it seems to work!

Thoughts: why are `Task` objects necessary at all?
- Pros: they streamline the run more by providing more precision to agents
- Cons: they could just be typed in the agent description, or fed as internal prompt upon agent run. Don't need to be separate objects.

> How to better adress the issue that CrewAI is trying to solve with Tasks:
- Make it easier to provide internal prompting to agents.